# 📘 Task Explanation: Softmax CUDA Kernel Implementation and Profiling

## 🎯 Objective
The objective of this task is to implement a **Softmax CUDA kernel** and then use **GPU profiling tools** to analyze its performance characteristics.  
Softmax is a **reduction-heavy and numerically sensitive** operator, making it an ideal case study for understanding GPU parallelism, memory access patterns, and performance bottlenecks.

This task emphasizes both **correctness** and **performance analysis**.

---

## 🧠 Background: What Is Softmax?
For an input vector \( x \in \mathbb{R}^D \), Softmax is defined as:
\[
\text{Softmax}(x_i) = \frac{e^{x_i}}{\sum_{j=1}^{D} e^{x_j}}
\]

In practice, a **numerically stable form** is used:
\[
\text{Softmax}(x_i) = \frac{e^{x_i - \max(x)}}{\sum_{j=1}^{D} e^{x_j - \max(x)}}
\]

Softmax involves:
- A **max reduction**
- A **sum reduction**
- Elementwise exponentiation and normalization

These operations make Softmax both **compute-intensive** and **memory-sensitive**.

---

## 🧩 Part A — Softmax CUDA Kernel

### Task
Design and implement a CUDA kernel for Softmax where:
- Each row (or vector) is processed independently
- Reductions (max and sum) are parallelized
- Numerical stability is ensured via the **subtract-max trick**

### Key Design Considerations
- How to map rows to thread blocks or warps
- How to implement max and sum reductions efficiently
- Whether to use shared memory or warp-level primitives
- Minimizing redundant global memory accesses

---

## 🧠 Correctness Requirements
- Use a numerically stable Softmax formulation
- Match a CPU reference implementation within tolerance
- Handle edge cases (large/small values, varying vector length)

---

## 🧩 Part B — Profiling the Softmax Kernel

### Task
Profile the Softmax CUDA kernel using GPU profiling tools such as:
- **Nsight Compute** (kernel-level analysis)
- (Optional) **Nsight Systems** (application-level timeline)

### What to Analyze
- Kernel execution time
- Warp execution efficiency
- Memory throughput and cache behavior
- Warp stalls vs memory stalls

---

## 📊 Key Metrics to Inspect
- Occupancy and active warps
- Stall reasons (e.g., memory dependency, execution dependency)
- Global memory load efficiency
- Shared memory usage and bank conflicts (if used)

---

## 🔍 Key Questions to Answer
- Is the kernel **memory-bound or compute-bound**?
- Which stage dominates runtime: max reduction, sum reduction, or normalization?
- Are reductions efficiently parallelized?
- Could kernel fusion or reduced memory traffic improve performance?

---

## 🧪 Deliverables
You should produce:
1. A Softmax CUDA kernel implementation
2. A CPU reference for correctness verification
3. Profiling reports (Nsight Compute)
4. A short analysis explaining:
   - Performance bottlenecks
   - Dominant stall reasons
   - Possible optimization directions

---

## 🎓 What You Learn from This Task
By completing this task, you will understand:
- How to implement reduction-heavy kernels on GPU
- Why numerical stability matters in GPU kernels
- How to interpret profiling metrics for real ML operators
- How Softmax kernels are optimized in practice

---

## 🚀 Relevance to ML Systems
Softmax is a core component in:
- Attention mechanisms
- Classification layers
- Transformer models

Efficient Softmax implementations are critical for:
- LLM training and inference
- Kernel fusion (e.g., FlashAttention)
- High-performance ML systems

---

## 🧠 Key Takeaway
> **Softmax combines numerical stability challenges with reduction-heavy computation, making it a perfect kernel for learning both CUDA optimization and GPU profiling.**


In [1]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Sat Jan 24 12:44:22 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P8       

In [ ]:
!apt-get update
!apt-get install -y cuda-toolkit-12-4

In [5]:
%%writefile softmax_profile_compare.cu
// Compare 3 CUDA softmax strategies with profiling:
//  (1) warp-per-row
//  (2) block-per-row
//  (3) multi-warp-per-row (tunable warpsPerRow)
//

#include <cuda_runtime.h>
#include <cstdio>
#include <cstdlib>
#include <cmath>
#include <vector>
#include <string>
#include <algorithm>
#include <random>
#include <iostream>

#define CUDA_CHECK(call) do {                                      \
  cudaError_t err = (call);                                        \
  if (err != cudaSuccess) {                                        \
    fprintf(stderr, "CUDA error %s:%d: %s\n",                      \
            __FILE__, __LINE__, cudaGetErrorString(err));          \
    std::exit(EXIT_FAILURE);                                       \
  }                                                                \
} while(0)

static inline int div_up(int a, int b) { return (a + b - 1) / b; }

// ======================= Warp-level reduce =======================

__device__ __forceinline__ float warpReduceMax(float v, unsigned mask = 0xffffffffu) {
  v = fmaxf(v, __shfl_down_sync(mask, v, 16));
  v = fmaxf(v, __shfl_down_sync(mask, v, 8));
  v = fmaxf(v, __shfl_down_sync(mask, v, 4));
  v = fmaxf(v, __shfl_down_sync(mask, v, 2));
  v = fmaxf(v, __shfl_down_sync(mask, v, 1));
  return v;
}

__device__ __forceinline__ float warpReduceSum(float v, unsigned mask = 0xffffffffu) {
  v += __shfl_down_sync(mask, v, 16);
  v += __shfl_down_sync(mask, v, 8);
  v += __shfl_down_sync(mask, v, 4);
  v += __shfl_down_sync(mask, v, 2);
  v += __shfl_down_sync(mask, v, 1);
  return v;
}

__device__ __forceinline__ float warpAllReduceMax(float v, unsigned mask = 0xffffffffu) {
  float m = warpReduceMax(v, mask);
  return __shfl_sync(mask, m, 0);
}

__device__ __forceinline__ float warpAllReduceSum(float v, unsigned mask = 0xffffffffu) {
  float s = warpReduceSum(v, mask);
  return __shfl_sync(mask, s, 0);
}

// ======================= Block-level reduce =======================
// Assumes blockDim.x is multiple of 32 and <= 1024
__device__ __forceinline__ float blockReduceMax(float v) {
  __shared__ float shm[32]; // up to 32 warps
  int lane = threadIdx.x & 31;
  int warp = threadIdx.x >> 5;
  int numWarps = (blockDim.x + 31) >> 5;

  v = warpReduceMax(v);
  if (lane == 0) shm[warp] = v;
  __syncthreads();

  float out = -INFINITY;
  if (warp == 0) {
    out = (lane < numWarps) ? shm[lane] : -INFINITY;
    out = warpReduceMax(out);
  }
  out = __shfl_sync(0xffffffffu, out, 0);
  return out;
}

__device__ __forceinline__ float blockReduceSum(float v) {
  __shared__ float shm[32];
  int lane = threadIdx.x & 31;
  int warp = threadIdx.x >> 5;
  int numWarps = (blockDim.x + 31) >> 5;

  v = warpReduceSum(v);
  if (lane == 0) shm[warp] = v;
  __syncthreads();

  float out = 0.0f;
  if (warp == 0) {
    out = (lane < numWarps) ? shm[lane] : 0.0f;
    out = warpReduceSum(out);
  }
  out = __shfl_sync(0xffffffffu, out, 0);
  return out;
}

// ============================================================
// (1) Warp-per-row kernel
// Mapping:
//   - 1 warp handles 1 row
//   - 1 block contains multiple warps => handles multiple rows
// ============================================================
__global__ void softmax_kernel_warp_per_row(const float* __restrict__ x,
                                            float* __restrict__ y,
                                            int B, int D) {
  int tid  = threadIdx.x;
  int lane = tid & 31;
  int warp = tid >> 5;
  int warpsPerBlock = blockDim.x >> 5;

  int row = blockIdx.x * warpsPerBlock + warp;
  if (row >= B) return;

  const float* row_x = x + row * D;
  float* row_y = y + row * D;

  float local_max = -INFINITY;
  for (int j = lane; j < D; j += 32) {
    local_max = fmaxf(local_max, row_x[j]);
  }
  float m = warpAllReduceMax(local_max);

  float local_sum = 0.0f;
  for (int j = lane; j < D; j += 32) {
    local_sum += expf(row_x[j] - m);
  }
  float s = warpAllReduceSum(local_sum);

  for (int j = lane; j < D; j += 32) {
    row_y[j] = expf(row_x[j] - m) / s;
  }
}

// ============================================================
// (2) Block-per-row kernel
// Mapping:
//   - 1 block handles 1 row
//   - threads stride across D
// ============================================================
__global__ void softmax_kernel_block_per_row(const float* __restrict__ x,
                                             float* __restrict__ y,
                                             int B, int D) {
  int row = blockIdx.x;
  if (row >= B) return;

  const float* row_x = x + row * D;
  float* row_y = y + row * D;

  float local_max = -INFINITY;
  for (int j = threadIdx.x; j < D; j += blockDim.x) {
    local_max = fmaxf(local_max, row_x[j]);
  }
  float m = blockReduceMax(local_max);

  float local_sum = 0.0f;
  for (int j = threadIdx.x; j < D; j += blockDim.x) {
    local_sum += expf(row_x[j] - m);
  }
  float s = blockReduceSum(local_sum);

  for (int j = threadIdx.x; j < D; j += blockDim.x) {
    row_y[j] = expf(row_x[j] - m) / s;
  }
}

// ============================================================
// (3) Multi-warp-per-row kernel (tunable)
// Mapping:
//   - 1 block handles 1 row
//   - blockDim.x = warpsPerRow * 32  (2/4/8/16 ...)
// Purpose:
//   - explore tradeoff between "too few threads" and "too much overhead"
//   - closer to attention-style blockwise softmax patterns
// ============================================================
__global__ void softmax_kernel_multiwarp_per_row(const float* __restrict__ x,
                                                 float* __restrict__ y,
                                                 int B, int D) {
  int row = blockIdx.x;
  if (row >= B) return;

  const float* row_x = x + row * D;
  float* row_y = y + row * D;

  float local_max = -INFINITY;
  for (int j = threadIdx.x; j < D; j += blockDim.x) {
    local_max = fmaxf(local_max, row_x[j]);
  }
  float m = blockReduceMax(local_max);

  float local_sum = 0.0f;
  for (int j = threadIdx.x; j < D; j += blockDim.x) {
    local_sum += expf(row_x[j] - m);
  }
  float s = blockReduceSum(local_sum);

  for (int j = threadIdx.x; j < D; j += blockDim.x) {
    row_y[j] = expf(row_x[j] - m) / s;
  }
}

// ======================= CPU reference (optional) =======================

static void softmax_cpu_ref(const std::vector<float>& x,
                            std::vector<float>& y,
                            int B, int D) {
  for (int b = 0; b < B; ++b) {
    const float* row_x = x.data() + b * D;
    float* row_y = y.data() + b * D;

    float m = -INFINITY;
    for (int j = 0; j < D; ++j) m = std::max(m, row_x[j]);

    double s = 0.0;
    for (int j = 0; j < D; ++j) s += std::exp(double(row_x[j] - m));

    for (int j = 0; j < D; ++j) row_y[j] = float(std::exp(double(row_x[j] - m)) / s);
  }
}

static bool check_close(const std::vector<float>& a,
                        const std::vector<float>& b,
                        float atol, float rtol,
                        int B, int D,
                        int max_report = 10) {
  int bad = 0;
  for (int i = 0; i < B * D; ++i) {
    float av = a[i], bv = b[i];
    float diff = std::fabs(av - bv);
    float tol = atol + rtol * std::fabs(bv);
    if (!(diff <= tol) || std::isnan(av) || std::isnan(bv)) {
      if (bad < max_report) {
        int row = i / D;
        int col = i % D;
        std::fprintf(stderr, "Mismatch at (row=%d,col=%d): gpu=%g ref=%g diff=%g tol=%g\n",
                     row, col, av, bv, diff, tol);
      }
      bad++;
      if (bad >= max_report) break;
    }
  }
  return bad == 0;
}

// ======================= Timing helper =======================

template <typename LaunchFn>
static float time_kernel_ms(LaunchFn launch, int warmup, int reps) {
  // Warmup
  for (int i = 0; i < warmup; ++i) launch();
  CUDA_CHECK(cudaDeviceSynchronize());

  cudaEvent_t start, stop;
  CUDA_CHECK(cudaEventCreate(&start));
  CUDA_CHECK(cudaEventCreate(&stop));

  CUDA_CHECK(cudaEventRecord(start));
  for (int i = 0; i < reps; ++i) launch();
  CUDA_CHECK(cudaEventRecord(stop));
  CUDA_CHECK(cudaEventSynchronize(stop));

  float ms = 0.0f;
  CUDA_CHECK(cudaEventElapsedTime(&ms, start, stop));

  CUDA_CHECK(cudaEventDestroy(start));
  CUDA_CHECK(cudaEventDestroy(stop));

  return ms / reps;
}

// ======================= CLI =======================

static int get_arg_int(int argc, char** argv, const char* name, int def) {
  for (int i = 1; i < argc - 1; ++i) {
    if (std::string(argv[i]) == name) return std::atoi(argv[i + 1]);
  }
  return def;
}
static int has_flag(int argc, char** argv, const char* name) {
  for (int i = 1; i < argc; ++i) if (std::string(argv[i]) == name) return 1;
  return 0;
}

int main(int argc, char** argv) {
  int B = get_arg_int(argc, argv, "--B", 4096);
  int D = get_arg_int(argc, argv, "--D", 1024);
  int reps = get_arg_int(argc, argv, "--reps", 200);
  int warmup = get_arg_int(argc, argv, "--warmup", 50);
  int check = get_arg_int(argc, argv, "--check", 0);

  int warpBlockWarps = get_arg_int(argc, argv, "--warp_block_warps", 8); // for warp-per-row
  int blockThreads   = get_arg_int(argc, argv, "--block_threads", 256);  // for block-per-row
  int mwWarps        = get_arg_int(argc, argv, "--mw_warps", 8);          // for multiwarp-per-row

  // sanitize
  warpBlockWarps = std::max(1, std::min(32, warpBlockWarps));
  blockThreads = std::max(32, std::min(1024, blockThreads));
  blockThreads = (blockThreads / 32) * 32; // force multiple of 32 for our blockReduce
  mwWarps = std::max(1, std::min(32, mwWarps));

  std::printf("B=%d D=%d reps=%d warmup=%d check=%d\n", B, D, reps, warmup, check);
  std::printf("Configs:\n");
  std::printf("  warp-per-row: warpsPerBlock=%d (block.x=%d)\n", warpBlockWarps, warpBlockWarps * 32);
  std::printf("  block-per-row: blockThreads=%d\n", blockThreads);
  std::printf("  multiwarp-per-row: warpsPerRow=%d (block.x=%d)\n", mwWarps, mwWarps * 32);

  size_t bytes = size_t(B) * size_t(D) * sizeof(float);
  std::vector<float> hx(B * D), hy(B * D), href;
  href.resize(B * D);

  // Random input with some variance (avoid all small values)
  std::mt19937 rng(123);
  std::normal_distribution<float> dist(0.0f, 5.0f);
  for (auto& v : hx) v = dist(rng);

  float *dx = nullptr, *dy = nullptr;
  CUDA_CHECK(cudaMalloc(&dx, bytes));
  CUDA_CHECK(cudaMalloc(&dy, bytes));
  CUDA_CHECK(cudaMemcpy(dx, hx.data(), bytes, cudaMemcpyHostToDevice));
  CUDA_CHECK(cudaMemset(dy, 0, bytes));

  // ---------------- Time kernels ----------------
  // (1) warp-per-row
  dim3 block1(warpBlockWarps * 32, 1, 1);
  dim3 grid1(div_up(B, warpBlockWarps), 1, 1);

  auto launch1 = [&]() {
    softmax_kernel_warp_per_row<<<grid1, block1>>>(dx, dy, B, D);
  };
  float ms1 = time_kernel_ms(launch1, warmup, reps);
  CUDA_CHECK(cudaGetLastError());

  // (2) block-per-row
  dim3 block2(blockThreads, 1, 1);
  dim3 grid2(B, 1, 1);
  auto launch2 = [&]() {
    softmax_kernel_block_per_row<<<grid2, block2>>>(dx, dy, B, D);
  };
  float ms2 = time_kernel_ms(launch2, warmup, reps);
  CUDA_CHECK(cudaGetLastError());

  // (3) multiwarp-per-row (tunable)
  dim3 block3(mwWarps * 32, 1, 1);
  dim3 grid3(B, 1, 1);
  auto launch3 = [&]() {
    softmax_kernel_multiwarp_per_row<<<grid3, block3>>>(dx, dy, B, D);
  };
  float ms3 = time_kernel_ms(launch3, warmup, reps);
  CUDA_CHECK(cudaGetLastError());

  // ---------------- Correctness (optional) ----------------
  if (check) {
    // run CPU reference
    softmax_cpu_ref(hx, href, B, D);

    // check each kernel once
    auto run_and_copy = [&](auto launch, const char* name) {
      CUDA_CHECK(cudaMemset(dy, 0, bytes));
      launch();
      CUDA_CHECK(cudaDeviceSynchronize());
      CUDA_CHECK(cudaMemcpy(hy.data(), dy, bytes, cudaMemcpyDeviceToHost));
      bool ok = check_close(hy, href, /*atol*/1e-5f, /*rtol*/1e-4f, B, D);
      std::printf("Check %-22s : %s\n", name, ok ? "PASS" : "FAIL");
    };
    run_and_copy(launch1, "warp-per-row");
    run_and_copy(launch2, "block-per-row");
    run_and_copy(launch3, "multiwarp-per-row");
  }

  // ---------------- Report ----------------
  auto report = [&](const char* name, float ms) {
    // Rough traffic estimate:
    // Read x (B*D floats) + write y (B*D floats) => 2 * bytes.
    // (softmax also re-reads x in 2nd/3rd loops; here we don't count that extra to keep "effective" bandwidth simple)
    double gb = (2.0 * double(bytes)) / 1e9;
    double gbps = gb / (double(ms) / 1e3);
    double ns_per_elem = (double(ms) * 1e6) / double(B) / double(D);
    std::printf("%-22s : %8.4f ms  |  ~%7.2f GB/s  |  %7.3f ns/elem\n", name, ms, gbps, ns_per_elem);
  };

  std::printf("\n=== Average time per launch ===\n");
  report("warp-per-row", ms1);
  report("block-per-row", ms2);
  report("multiwarp-per-row", ms3);

  // Suggest next steps for deep profiling
  std::printf("\n=== Nsight Compute tips ===\n");
  std::printf("1) Run full profile for all kernels:\n");
  std::printf("   ncu --set full --kernel-name regex:softmax_kernel_.* ./softmax_prof --B %d --D %d --reps 50 --warmup 10\n", B, D);
  std::printf("2) Focus on memory & math throughput:\n");
  std::printf("   ncu --metrics sm__throughput.avg.pct_of_peak_sustained_elapsed,dram__throughput.avg.pct_of_peak_sustained_elapsed,smsp__sass_thread_inst_executed_op_fadd_pred_on.sum,smsp__sass_thread_inst_executed_op_ffma_pred_on.sum ./softmax_prof --B %d --D %d --reps 50 --warmup 10\n", B, D);
  std::printf("3) Look at stalls (scheduler / memory dependency):\n");
  std::printf("   ncu --section \"Warp State Statistics\" --section \"Scheduler Statistics\" --section \"Memory Workload Analysis\" ./softmax_prof --B %d --D %d --reps 50 --warmup 10\n", B, D);

  CUDA_CHECK(cudaFree(dx));
  CUDA_CHECK(cudaFree(dy));
  return 0;
}


Writing softmax_profile_compare.cu


In [7]:
!nvcc -arch=sm_75 softmax_profile_compare.cu -o softmax_prof

softmax_profile_compare.cu(279): warning #177-D: function "has_flag" was declared but never referenced
  static int has_flag(int argc, char** argv, const char* name) {
             ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"



In [8]:
# D 小,B 大(warp-per-row 通常赢）
!./softmax_prof --B 65536 --D 128  --reps 300 --warmup 50

B=65536 D=128 reps=300 warmup=50 check=0
Configs:
  warp-per-row: warpsPerBlock=8 (block.x=256)
  block-per-row: blockThreads=256
  multiwarp-per-row: warpsPerRow=8 (block.x=256)

=== Average time per launch ===
warp-per-row           :   0.2871 ms  |  ~ 233.75 GB/s  |    0.034 ns/elem
block-per-row          :   0.9043 ms  |  ~  74.21 GB/s  |    0.108 ns/elem
multiwarp-per-row      :   0.7211 ms  |  ~  93.06 GB/s  |    0.086 ns/elem

=== Nsight Compute tips ===
1) Run full profile for all kernels:
   ncu --set full --kernel-name regex:softmax_kernel_.* ./softmax_prof --B 65536 --D 128 --reps 50 --warmup 10
2) Focus on memory & math throughput:
   ncu --metrics sm__throughput.avg.pct_of_peak_sustained_elapsed,dram__throughput.avg.pct_of_peak_sustained_elapsed,smsp__sass_thread_inst_executed_op_fadd_pred_on.sum,smsp__sass_thread_inst_executed_op_ffma_pred_on.sum ./softmax_prof --B 65536 --D 128 --reps 50 --warmup 10
3) Look at stalls (scheduler / memory dependency):
   ncu --section "War

In [9]:
# D 中等（multi-warp-per-row 往往更容易成为最优）
!./softmax_prof --B 16384 --D 512  --reps 200 --warmup 50 --mw_warps 4
!./softmax_prof --B 16384 --D 512  --reps 200 --warmup 50 --mw_warps 8
!./softmax_prof --B 16384 --D 512  --reps 200 --warmup 50 --mw_warps 16

B=16384 D=512 reps=200 warmup=50 check=0
Configs:
  warp-per-row: warpsPerBlock=8 (block.x=256)
  block-per-row: blockThreads=256
  multiwarp-per-row: warpsPerRow=4 (block.x=128)

=== Average time per launch ===
warp-per-row           :   0.3332 ms  |  ~ 201.43 GB/s  |    0.040 ns/elem
block-per-row          :   0.5817 ms  |  ~ 115.37 GB/s  |    0.069 ns/elem
multiwarp-per-row      :   0.4076 ms  |  ~ 164.65 GB/s  |    0.049 ns/elem

=== Nsight Compute tips ===
1) Run full profile for all kernels:
   ncu --set full --kernel-name regex:softmax_kernel_.* ./softmax_prof --B 16384 --D 512 --reps 50 --warmup 10
2) Focus on memory & math throughput:
   ncu --metrics sm__throughput.avg.pct_of_peak_sustained_elapsed,dram__throughput.avg.pct_of_peak_sustained_elapsed,smsp__sass_thread_inst_executed_op_fadd_pred_on.sum,smsp__sass_thread_inst_executed_op_ffma_pred_on.sum ./softmax_prof --B 16384 --D 512 --reps 50 --warmup 10
3) Look at stalls (scheduler / memory dependency):
   ncu --section "War

In [10]:
# D 大（block-per-row 更稳）
!./softmax_prof --B 4096 --D 4096 --reps 120 --warmup 20 --block_threads 256
!./softmax_prof --B 4096 --D 4096 --reps 120 --warmup 20 --block_threads 512

B=4096 D=4096 reps=120 warmup=20 check=0
Configs:
  warp-per-row: warpsPerBlock=8 (block.x=256)
  block-per-row: blockThreads=256
  multiwarp-per-row: warpsPerRow=8 (block.x=256)

=== Average time per launch ===
warp-per-row           :   1.2928 ms  |  ~ 103.82 GB/s  |    0.077 ns/elem
block-per-row          :   1.0331 ms  |  ~ 129.91 GB/s  |    0.062 ns/elem
multiwarp-per-row      :   0.7535 ms  |  ~ 178.12 GB/s  |    0.045 ns/elem

=== Nsight Compute tips ===
1) Run full profile for all kernels:
   ncu --set full --kernel-name regex:softmax_kernel_.* ./softmax_prof --B 4096 --D 4096 --reps 50 --warmup 10
2) Focus on memory & math throughput:
   ncu --metrics sm__throughput.avg.pct_of_peak_sustained_elapsed,dram__throughput.avg.pct_of_peak_sustained_elapsed,smsp__sass_thread_inst_executed_op_fadd_pred_on.sum,smsp__sass_thread_inst_executed_op_ffma_pred_on.sum ./softmax_prof --B 4096 --D 4096 --reps 50 --warmup 10
3) Look at stalls (scheduler / memory dependency):
   ncu --section "War